## Predict Bad_flag using Decision Tree

Dataset: Lending Club loan dataset from Kaggle

https://www.kaggle.com/wordsforthewise/lending-club

## Libraries

In [1]:
%matplotlib inline ## no need of plt.show()
%reload_ext autoreload
%autoreload 2

UsageError: unrecognized arguments: ## no need of plt.show()


In [2]:
# basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import re, string, os, sys
from datetime import datetime
import warnings # To ignore any warnings (Good to have)
warnings.filterwarnings("ignore")

# machine learning
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import roc_auc_score,roc_curve,classification_report,confusion_matrix,auc,f1_score, make_scorer,average_precision_score,precision_recall_curve # metrics
from sklearn.model_selection import GridSearchCV

## Import Data

In [3]:
path="D:/Upskilling/data challenge/chime/loan.csv/" # update this as path where you keep your data files while runnng notebook

In [4]:
missing_values = ["n/a", "na", "--","NA","NAN","na","nan"] # checking NAs, NANs if present as string in dataframe 
df = pd.read_csv(path+"loan.csv",na_values = missing_values)

### Data Summary

In [5]:
df_subset = df.sample(n=50000,random_state=1).copy() # sample 50K rows to lower computing time
df_subset = df_subset.reset_index(drop=True) # reset index, drop index becoming column

## Feature Engineering

In [6]:
# Convert employee length from string to numeric
df_subset['emp_length'].unique()
df_subset['emp_length_int'] =df_subset['emp_length'].str.replace('\+ years','')
df_subset['emp_length_int'] =df_subset['emp_length_int'].str.replace('< 1 year',str(0))
df_subset['emp_length_int'] =df_subset['emp_length_int'].str.replace(' years','')
df_subset['emp_length_int'] =df_subset['emp_length_int'].str.replace(' year','')
df_subset['emp_length_int'].unique()
df_subset['emp_length_int'] =pd.to_numeric(df_subset['emp_length_int'])
df_subset['emp_length_int'].unique(),type(df_subset['emp_length_int'][0])
# Convert Earliest Credit Line date to Month since Earliest Credit Line
df_subset['earliest_cr_line'][0]
pd.to_datetime(df_subset['earliest_cr_line'][0],format='%b-%Y')
df_subset['earliest_cr_line_date']=pd.to_datetime(df_subset['earliest_cr_line'],format='%b-%Y')
type(df_subset['earliest_cr_line_date'][0])
df_subset['months_since_earliest_cr_line'] = round((pd.to_datetime('2017-12-01')- df_subset['earliest_cr_line_date'])/np.timedelta64(1,'M'))
df_subset['months_since_earliest_cr_line'].describe()
df_subset[['earliest_cr_line','earliest_cr_line_date','months_since_earliest_cr_line']].head()

,earliest_cr_line,earliest_cr_line_date,months_since_earliest_cr_line
0,Jan-1983,1983-01-01,419.0
1,Oct-1991,1991-10-01,314.0
2,May-1989,1989-05-01,343.0
3,Feb-2008,2008-02-01,118.0
4,Feb-1992,1992-02-01,310.0


### Data PreProcessing

In [7]:
# Remove Id related columns
df_subset_x =df_subset.drop(["id","member_id","url"], axis =1).select_dtypes(include=['float64','int64'])
# Imputing missing values
df_subset_x.fillna(value=-999,inplace=True)

In [8]:
# Converting Depending variable as Default Indicator (1/0)
df_subset['loan_status'].unique()
df_subset['loan_status'].value_counts() /df_subset['loan_status'].count()
df_subset['is_default'] = np.where(df_subset['loan_status'].isin(['Charged Off','Default',
                                                                  'Does not meet the credit policy. Status:Charged Off',
                                                                 'Late (16-30 days)']),1,0)
df_subset_y =df_subset['is_default']
df_subset_y.head()

0    0
1    0
2    0
3    0
4    0
Name: is_default, dtype: int32

### Model building

In [9]:
X = df_subset_x.values # independent variable
Y = df_subset_y.values # target variable
feature_cols=df_subset_x.columns # independent variable columns

In [10]:
X.shape, Y.shape

((50000, 108), (50000,))

In [11]:
# Splitting the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split( 
X, Y, test_size = 0.3, random_state = 100)

In [12]:
y_train, y_test

(array([0, 0, 0, ..., 0, 0, 0]), array([0, 1, 0, ..., 0, 0, 1]))

### Base Decision Tree Model

In [13]:
clf = DecisionTreeClassifier(random_state = 0)
# Train the model on the data
clf.fit(X_train, y_train)
# Predict labels of seen (train) data
y_pred_train = clf.predict(X_train) # Prediction as 0 or 1
y_pred_prob_train = clf.predict_proba(X_train) # Prediction as probability of default
# Predict labels of unseen (test) data
y_pred_test = clf.predict(X_test)
y_pred_prob_test = clf.predict_proba(X_test)

df = pd.DataFrame({'y_test':y_test,'y_pred':y_pred_test,'y_pred_prob':y_pred_prob_test[:,1]})
df.head()

,y_test,y_pred,y_pred_prob
0,0,0,0.0
1,1,1,1.0
2,0,0,0.0
3,1,1,1.0
4,0,0,0.0


### Scoring criteria
F1 score as objective and auc pr (Average Precision Score) as auxilary metrix, final results are being evaluate on F1 Score.

In [14]:
def scores(y_actual,y_pred,y_pred_prob):
    AUC=roc_auc_score(y_actual, y_pred_prob[:,1])
    PR_AUC = average_precision_score(y_actual, y_pred_prob[:,1])
    F1_scr = f1_score(y_actual, y_pred)
    
    scr_df = pd.DataFrame({
    'AUC': [AUC],
    'PR_AUC': [PR_AUC],
    'F1_scr': [F1_scr]})
    return scr_df

In [15]:
scores(y_train,y_pred_train,y_pred_prob_train)

,AUC,PR_AUC,F1_scr
0,1.0,1.0,1.0


In [16]:
scores(y_test,y_pred_test,y_pred_prob_test)

,AUC,PR_AUC,F1_scr
0,0.979659,0.929263,0.961832


### Optimizing Decision Tree with Hyperparameter Tuning

In [23]:
params = {
    'max_depth': [2, 3, 5, 10, 20],
    'min_samples_leaf': [5, 10, 20, 50, 100],
    'criterion': ["gini", "entropy"]
}

In [24]:
f1 = make_scorer(f1_score, average='micro')
# Instantiate the grid search model
m=DecisionTreeClassifier()
clf = GridSearchCV(m, params, n_jobs=5, 
                   cv=StratifiedKFold(n_splits=5, shuffle=True, random_state = 0).split(X_train, y_train), 
                   scoring=f1,
                   verbose=2, refit=True)

In [25]:
%%time
clf.fit(X_train, y_train) # code takes long time

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Wall time: 57.6 s


GridSearchCV(cv=<generator object _BaseKFold.split at 0x0000020DA1F7E150>,
             estimator=DecisionTreeClassifier(), n_jobs=5,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 5, 10, 20],
                         'min_samples_leaf': [5, 10, 20, 50, 100]},
             scoring=make_scorer(f1_score, average=micro), verbose=2)

In [26]:
#print('\n Best estimator:')
print("best_estimator:",clf.best_estimator_)
#print('\n Best hyperparameters:')
print("best_params:", clf.best_params_)
clf.best_score_

best_estimator: DecisionTreeClassifier(criterion='entropy', max_depth=20, min_samples_leaf=10)
best_params: {'criterion': 'entropy', 'max_depth': 20, 'min_samples_leaf': 10}


0.995857142857143

In [27]:
# Make an instance of the Optimized Model
clf_opt = DecisionTreeClassifier(max_depth=10, min_samples_leaf=50, criterion='gini',
                             random_state = 0)
# Train the model on the data
clf_opt.fit(X_train, y_train)

y_pred_train_2 = clf_opt.predict(X_train)
y_pred_prob_train_2 = clf_opt.predict_proba(X_train)

y_pred_test_2 = clf.predict(X_test)
y_pred_prob_test_2 = clf.predict_proba(X_test)

#my_array = np.column_stack((y_test,y_pred_prob[:,1],y_pred)).shape

df_opt = pd.DataFrame({'y_test':y_test,'y_pred':y_pred_test_2,'y_pred_prob':y_pred_prob_test_2[:,1]})
df_opt.head()

,y_test,y_pred,y_pred_prob
0,0,0,0.0
1,1,1,1.0
2,0,0,0.0
3,1,1,1.0
4,0,0,0.0


In [28]:
scores(y_test,y_pred_test_2,y_pred_prob_test_2)

,AUC,PR_AUC,F1_scr
0,0.983268,0.970312,0.979474


### Final Decision Tree

In [29]:
from sklearn.tree import export_graphviz
export_graphviz(clf_opt, out_file='tree_limited.dot', feature_names = feature_cols,max_depth=5,
                class_names = ['0','1'],
                rounded = True, proportion = True, precision = 2, filled = True)

In [30]:
!dot -Tpng tree_limited.dot -o tree_limited.png -Gdpi=600

In [31]:
from IPython.display import Image
Image(filename = 'tree_limited.png')

### End